In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from functools import reduce
from itertools import product
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.regression.linear_model import yule_walker
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.api import SARIMAX

from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing
Making dummies for weather

This part contains constants, like city names, main russian categories etc.

In [ ]:
REGIONS = ['kyiv','chernihiv', 'sumy','kharkiv',
           'izyum','donetsk', 'melitopol', 'kherson']
NORTH_REGIONS = REGIONS[:3]
MAIN_REGIONS = REGIONS[3:]
MAIN_CATEGORIES = ['soldiers', 'tanks', 'armveh']
WEAPON_AFRAID = ['drones', 'boats', 'mlrs', 'artillery']
WEATHER_METRICS = ['temp', 'wind']

This function reads the data for one particular region.

In [ ]:
def get_region_weather(region=None, index_col='date'):
    '''
    Reads weather data for a specific city.
    Adds prefix to specify the region
    '''
    filename = filename=f'/content/drive/Shareddrives/war-statistics/{region.lower()}_weather_stats.xlsx'
    data = pd.read_excel(filename, index_col=index_col)
    data.index = pd.to_datetime(data.index, dayfirst=True)
    data = data[data['temp'].notna()]
    
    data = data.add_prefix(f'{region.lower()}_')
    data = data.astype('int16')
    return data

data = get_region_weather(region='sumy')
data.columns

Index(['sumy_temp', 'sumy_pres', 'sumy_cloud', 'sumy_wind'], dtype='object')

This function merges weather data into 1 complete dataset.

In [ ]:
def merge_regions_weather():
    '''
    Extracts and merges weather data from a set of regions.
    '''
    weather = []
    for region in REGIONS:
        weather.append(get_region_weather(region=region))
    
    merged_weather = reduce(lambda left, right:
                            pd.merge(left, right, on=['date'], how='inner'), weather)
    return merged_weather

all_weather = merge_regions_weather()
all_weather.shape

(98, 32)

data - **FULL** dataset

past - past data (i.e. past, known, historical data)

future - future data for **predictions** (i.e. future dates)

cumm - cummulative data with losses.

diff - day-difference data (day-increase, delta)

control_col in read_data() params is the column in which we seek for the first appearence of NaN, and split into past and future dataframes.

### Descriptive statistics

This function reads the russian casualties data.

In [ ]:
def read_data(filename='/content/drive/Shareddrives/war-statistics/war-data.xlsx',
              control_col='soldiers', index_col='date'):
    """
    Reads the data from excel file to 3 dataframes:
    cumm: past data
    diff: day-increase
    future: future data
    """
    data = pd.read_excel(filename, index_col=index_col)
    
    data.index = pd.to_datetime(data.index, dayfirst=True)
    past, future = data[data[control_col].notnull()], data[data[control_col].isnull()].fillna(-1)
    past = past.astype('int32')
    cumm, diff = past.copy(), past
    diff = diff.diff()[1:].astype('int32')
    return cumm, diff, future

cumm, diff, future = read_data()
cumm.shape, diff.shape, future.shape
diff.iloc[-5:,:]

,soldiers,tanks,armveh,artillery,mlrs,airdef,aircraft,helicopters,drones,vehicles,boats,specequip
date,,,,,,,,,,,,
2022-05-18,400,16,34,8,1,1,1,0,5,28,0,0
2022-05-19,200,3,20,9,0,2,1,0,14,20,0,0
2022-05-20,200,9,27,1,1,0,1,1,5,5,0,0
2022-05-21,150,15,26,0,1,0,0,1,2,16,0,0
2022-05-22,200,7,25,3,0,0,0,1,8,16,0,0


This function generates discriptive statistics for russian losses.

In [ ]:
def get_losses_descriptive_stats(diff):
    first, second = [], []

    for idx, col in enumerate(diff.columns):
        if idx % 2:
            first.append((
            col, 'mean ' + str(round(diff[col].mean())), 'max ' + str(diff[col].max())
        ))
        else:
            second.append((
            col, 'mean ' + str(round(diff[col].mean())), 'max ' + str(diff[col].max())
        ))
    
    return pd.DataFrame(zip(first, second), columns=['1', '2'])

d_s = get_losses_descriptive_stats(diff)
d_s.to_excel('/content/drive/Shareddrives/war-statistics/Util/descriptive_stats_losses.xlsx')
d_s

,1,2
0,"(tanks, mean 15, max 66)","(soldiers, mean 333, max 3160)"
1,"(artillery, mean 7, max 49)","(armveh, mean 36, max 516)"
2,"(airdef, mean 1, max 8)","(mlrs, mean 2, max 17)"
3,"(helicopters, mean 2, max 20)","(aircraft, mean 2, max 16)"
4,"(vehicles, mean 25, max 261)","(drones, mean 5, max 26)"
5,"(specequip, mean 0, max 10)","(boats, mean 0, max 2)"


This function generates discriptive statistics for weather data among all 8 regions.

In [ ]:
def get_weather_descriptive_stats(data):
    descriptive_stats = []
    for region in REGIONS:

        for metric in WEATHER_METRICS:

            descriptive_stats.append((
                region, metric,
                round(data[f'{region}_{metric}'].mean(), 2),
                list(map(int, (data[f'{region}_{metric}'].min(),
                data[f'{region}_{metric}'].max())))
            ))
    return '\n'.join(map(lambda x: f'{x[0].title()} {x[1]}, mean={x[2]}, min-max {x[3]}', descriptive_stats))

d_s = get_weather_descriptive_stats(all_weather)
d_s

'Kyiv temp, mean=4.91, min-max [-4, 20]\nKyiv wind, mean=3.22, min-max [0, 10]\nChernihiv temp, mean=4.13, min-max [-3, 20]\nChernihiv wind, mean=3.02, min-max [0, 13]\nSumy temp, mean=4.27, min-max [-5, 18]\nSumy wind, mean=3.27, min-max [0, 10]\nKharkiv temp, mean=9.0, min-max [-10, 23]\nKharkiv wind, mean=5.38, min-max [1, 12]\nIzyum temp, mean=11.12, min-max [-9, 25]\nIzyum wind, mean=3.52, min-max [1, 7]\nDonetsk temp, mean=11.84, min-max [-4, 27]\nDonetsk wind, mean=3.94, min-max [1, 8]\nMelitopol temp, mean=12.58, min-max [-4, 27]\nMelitopol wind, mean=4.37, min-max [0, 9]\nKherson temp, mean=12.23, min-max [-4, 25]\nKherson wind, mean=5.16, min-max [1, 14]'

## merge_all()

This function is needed to combine weather and casualties dataframe

In [ ]:
def merge_all(weather, diff):
    weather = weather.loc['2022-02-25':,:]
    data = pd.concat([diff, weather], axis=1)
    data = sm.add_constant(data)
    return data

data = merge_all(all_weather, diff)
data.shape

(87, 45)

# **Visualizations**


**plot_all_descriptive()**

This function prints descriptive statistics for russian casualties

In [ ]:
def plot_all_descriptive(data=None):
    """
    Creates 14 side-by-side plots.
    Each with its own category of losts.
    """
    df = data.copy()
    counter, cols = 0, 2
    figure = make_subplots(
        rows=7, cols=2,
        subplot_titles=df.columns)

    for category in df.columns:
        row, col = counter // cols + 1, counter % cols + 1
        fig = go.Box(
            x=df[category], name=''
        )
        counter += 1
        figure.add_trace(fig, row=row, col=col)

    figure.update_layout(height=900, width=1100, showlegend=False,
                         title_text=f'Categories daily increase descriptive')
    return figure

figure = plot_all_descriptive(data=diff)
figure.show()

**plot_all_descriptive_weather()**

This function prints descriptive statistics for weather data

In [ ]:
def plot_all_descriptive_weather(data=None):
    """
    Creates 14 side-by-side plots.
    Each with its own category of losts.
    """
    df = data.copy()
    counter, cols = 0, 2
    row_titles = [region.title() for region in REGIONS]
    figure = make_subplots(
        rows=8, cols=2, column_titles=['temperature', 'wind'], row_titles=row_titles
        )

    colors = {'wind': 'black', 'temp': 'red'}
    for region in REGIONS:
        
        for metric in WEATHER_METRICS:
            row, col = counter // cols + 1, counter % cols + 1
            idx = '2022-04-20' if region in NORTH_REGIONS else '2022-05-22'
            fig = go.Box(
                x=df[f'{region}_{metric}'][:idx], name='', marker_color=colors[metric]
            )

            figure.add_trace(fig, row=row, col=col)
            counter += 1

    figure.update_layout(height=900, width=1100, showlegend=False,
                         title_text=f'Main weather metrics descriptive')
    return figure

figure = plot_all_descriptive_weather(data=all_weather)
figure.show()

# **plot_category**()
Function **plot_category**() and it's usage.

Shows past-historical data.

In [ ]:
def plot_category(data=None, log_y=False, show_category=None, is_cumm=True):
    """
    Creates bar chart graph with one selected category.
    """
    title = f'Cummulative losses of {show_category}' if is_cumm else f'One day losses of {show_category}'
    fig = px.line(x=data.index, y=data[show_category], log_y=log_y, markers=True,
                  title=title)
    fig.update_xaxes(title='Date', showgrid=False)
    fig.update_yaxes(title='losses', showgrid=False)
    if not is_cumm:
        
        mean, std = data[show_category].mean(), data[show_category].std()
        x_range = [data.index.min(), data.index.max()]

        fig.add_hline(y=mean, line_width=3, line_dash="dash", line_color="red",
                      annotation_text="mean", annotation=dict(font_size=20))
        fig.add_hrect(y0=mean-2*std, y1=mean+2*std, line_width=1, fillcolor="red",
                  opacity=0.3, annotation_text="2 sigma neighbourhood", annotation_position="top right",
                  annotation=dict(font_size=20))

    
    figure = go.Figure(fig)
    
    return figure


fig = plot_category(data=cumm, show_category='soldiers', is_cumm=True)
fig.show()

# **plot_all()**

Plot first difference plots for russian losses

In [ ]:
from plotly.subplots import make_subplots

def plot_all(data=None):
    """
    Creates 14 side-by-side plots.
    Each with its own category of losts.
    """
    df = data.copy()
    counter, cols = 0, 2
    figure = make_subplots(
        rows=7, cols=2,
        subplot_titles=df.columns)

    for category in df.columns:
        row, col = counter // cols + 1, counter % cols + 1
        fig = go.Scatter(
            x=df.index, y=df[category], name=category
        )
        counter += 1
        figure.add_trace(fig, row=row, col=col)

    figure.update_layout(height=900, width=1100, showlegend=False,
                         title_text=f'Categories daily increase')
    return figure

figure = plot_all(data=diff)
figure.show()

# **plot_all_weather()**

Print weather data with smoothing rolling window at the backfront

In [ ]:
def plot_all_weather(data=None):
    """
    Creates 14 side-by-side plots.
    Each with its own category of losts.
    """
    df = data.copy()
    counter, cols = 0, 2
    row_titles = [region.title() for region in REGIONS]
    figure = make_subplots(
        rows=8, cols=2, column_titles=['temperature', 'wind'], row_titles=row_titles
        )

    colors = {'wind': 'black', 'temp': 'red'}
    inv_colors = {'wind': 'red', 'temp': 'black'}
    for region in REGIONS:

        for metric in WEATHER_METRICS:
            # idx = 68 if region in NORTH_REGIONS else len(df)
            if region in NORTH_REGIONS:
                df[f'{region}_{metric}']['2022-04-20':] = None


            row, col = counter // cols + 1, counter % cols + 1
            name = region.title() + ' '+ metric
            fig = go.Scatter(
                x=df.index, y=df[f'{region}_{metric}'], name=name,
                line=dict(color=colors[metric], width=4)
            )
            figure.add_trace(fig, row=row, col=col)
            
            fig = go.Scatter(x=df.index, y=df[f'{region}_{metric}'].rolling(9).mean(),
                             line=dict(color=inv_colors[metric], width=3, dash='dot'), name=f'{metric} MA[9]')
            figure.add_trace(fig, row=row, col=col)

            counter += 1

    figure.update_layout(height=900, width=1100, showlegend=False,
                         title_text=f'Main daily weather metrics (with smoothing MA[9])')
    return figure

figure = plot_all_weather(data=all_weather)
figure.show()

# Weighting the weather data by air alarms.

In [ ]:
def gen_weights():
    '''
    '''
    alarms_hours = {
    "kyiv_reg": 182,
    "sumy_reg": 115,
    "chernihiv_reg": 125,
    "kharkiv_reg": 343,
    "donetsk_reg": 244,
    "melitopol_reg": 235,
    "kherson_reg": 2
    }

    total_alarms = sum(alarms_hours.values())

    for key in alarms_hours.keys():
        alarms_hours[key] /= total_alarms

    return alarms_hours

AIR_ALARM = gen_weights()
AIR_ALARM

{'chernihiv_reg': 0.10032102728731943,
 'donetsk_reg': 0.1958266452648475,
 'kharkiv_reg': 0.2752808988764045,
 'kherson_reg': 0.0016051364365971107,
 'kyiv_reg': 0.14606741573033707,
 'melitopol_reg': 0.18860353130016053,
 'sumy_reg': 0.09229534510433386}

# Merging weather data

In [ ]:
def weather_data(region=None, index_col='date'):
    '''
    Reads weather data for a specific city.
    Adds prefix to specify the region
    '''
    filename = filename=f'/content/drive/Shareddrives/war-statistics/{region.lower()}_weather_stats.xlsx'
    
    data = pd.read_excel(filename, index_col=index_col)
    data.index = pd.to_datetime(data.index, dayfirst=True)

    data = data[~ (pd.isna(data.temp))]
    data = data.add_prefix(f'{region.lower()}_')
    return data

def all_regions_weather():
    '''
    Extracts and merges weather data from a set of regions.
    '''
    regions = ['kyiv','sumy','izyum','kharkiv','kherson',
                'melitopol','donetsk','chernihiv']
    weather = []
    for region in regions:
        weather.append(weather_data(region=region))
    
    weath_merged = reduce(lambda left,right: pd.merge(left,right,on=['date'],
                                            how='inner'), weather)
    
    return weath_merged

# Smoothing temperature

In [ ]:
def generate_moving_average(data=None, region=None, category=None, w_size=5):
    '''
    Adds a moving average data representation with a specific lag.
    
    data: data
    region: region name
    category: weather category
    w_size: size of the window in the MA
    '''
    data[f'MA_{region}_{category}'] = data[
        f"{region}_{category}"].rolling(w_size).mean()


def region_moving_average(data=None, region=None, w_size=9, inplace=False):
    '''
    Adds a moving average data representation with a specific lag
    for each category (currently):
        temperature
        cloudness
        wind
    
    data: data
    region: region name
    w_size: size of the window in the MA
    inplace: if a new object is created. Default=False
    '''
    categories = ['temp', 'wind']

    if inplace:
        df = data.copy()
        for cat in categories:
            generate_moving_average(data=df, 
                                    region=region, 
                                    category=cat,
                                    w_size=w_size)

    else:
        for cat in categories:
            generate_moving_average(data=data, 
                                    region=region, 
                                    category=cat,
                                    w_size=w_size)

    if inplace:
        return df

def all_data_moving_average():
    '''
    Applies region_moving_average() for each region in dataset.
    '''
    all_w_data = all_regions_weather()

    for region in ['kyiv','sumy','izyum','kharkiv','kherson',
                'melitopol','donetsk','chernihiv']:
        region_moving_average(data=all_w_data, region=region)
    
    return all_w_data

k_weather = weather_data(region="kharkiv")
region_moving_average(data=k_weather, region="kharkiv")
k_weather.head(5)

,kharkiv_temp,kharkiv_pres,kharkiv_cloud,kharkiv_wind,MA_kharkiv_temp,MA_kharkiv_wind
date,,,,,,
2022-02-14,2.0,759.0,0.0,6.0,NaN,NaN
2022-02-15,3.0,759.0,0.0,3.0,NaN,NaN
2022-02-16,0.0,755.0,3.0,3.0,NaN,NaN
2022-02-17,4.0,742.0,3.0,6.0,NaN,NaN
2022-02-18,4.0,739.0,3.0,5.0,NaN,NaN


# Merge weightened weather data

In [ ]:
# merge weather data with war data
def weighten_data():
    '''
    Weightens data relying on of air alarms distribution (in hours) on each region.

    Parameter
    '''
    weather = all_data_moving_average()
    weather["active_north"] = np.where(weather.index < "2022-04-05", 1, 0)
    combined_data = pd.DataFrame()
    combined_data.index = weather.index
    combined_data.north = weather["active_north"]

    for indic in ["temp", "wind"]:
        combined_data[indic] = AIR_ALARM["kharkiv_reg"]*(weather[f"MA_kharkiv_{indic}"] + weather[f"MA_izyum_{indic}"])
        
        for reg in ['kherson', 'melitopol','donetsk']:
            combined_data[indic] += AIR_ALARM[f"{reg}_reg"]*weather[f"MA_{reg}_{indic}"]
        
        for reg in ['kyiv', 'sumy', 'chernihiv']:
            combined_data[indic] += AIR_ALARM[f"{reg}_reg"]*weather[f"MA_{reg}_{indic}"]*weather["active_north"]
        

    for indic in ["pres", "cloud"]:
        combined_data[indic] = AIR_ALARM["kharkiv_reg"]*(weather[f"kharkiv_{indic}"] + weather[f"izyum_{indic}"])
        
        for reg in ['kherson', 'melitopol','donetsk']:
            combined_data[indic] += AIR_ALARM[f"{reg}_reg"]*weather[f"{reg}_{indic}"]
        
        for reg in ['kyiv', 'sumy', 'chernihiv']:
            combined_data[indic] += AIR_ALARM[f"{reg}_reg"]*weather[f"{reg}_{indic}"]*weather["active_north"]

    return combined_data

weather = weighten_data().loc['2022-02-25':,:]

# Merged

In [ ]:
# merge weather data with war data
weather = weighten_data()
w_diff = weather.dropna().diff()[1:]
weather["temp"], weather["pres"], weather["wind"] = w_diff["temp"], w_diff["pres"], w_diff["wind"]
war_weath = pd.merge(diff, weather, how="inner", on = 'date').dropna()

war_weath.describe()

,soldiers,tanks,armveh,artillery,mlrs,airdef,aircraft,helicopters,drones,vehicles,boats,specequip,temp,wind,pres,cloud
count,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000
mean,332.758621,14.609195,36.103448,6.885057,2.310345,1.068966,2.264368,1.908046,5.402299,25.218391,0.149425,0.494253,0.094799,-0.015255,-3.180421,1.797827
std,475.429426,10.149300,56.927337,7.725562,3.218082,1.597914,2.830317,3.476272,5.488804,34.014164,0.445357,1.354352,0.745020,0.324285,27.560911,1.052080
min,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.522561,-2.148564,-253.989567,0.000000
25%,158.000000,8.000000,17.500000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,9.000000,0.000000,0.000000,-0.361334,-0.142768,-2.820225,1.090690
50%,200.000000,13.000000,27.000000,5.000000,1.000000,1.000000,2.000000,1.000000,3.000000,19.000000,0.000000,0.000000,0.059836,-0.019083,-0.110754,1.793740
75%,300.000000,17.000000,38.500000,8.500000,3.000000,2.000000,3.000000,2.000000,8.500000,27.500000,0.000000,0.500000,0.461298,0.150571,2.969904,2.532103
max,3160.000000,66.000000,516.000000,49.000000,17.000000,8.000000,16.000000,20.000000,26.000000,261.000000,2.000000,10.000000,2.249153,0.668807,11.148475,3.825843


# Check stationarity

In [ ]:
def category_stationarity(data=None, category=None):
    '''
    Checks if the given category is stationary
    '''
    ADF_result = adfuller(data[category])

    print(f"Results for {category}")
    print(f'ADF Statistic: {ADF_result[0]}') 
    print(f'p-value: {ADF_result[1]}')
    return ADF_result[1]

x = []
y = []

for cat in war_weath.columns:
    y.append(category_stationarity(data=war_weath, category=cat))
    x.append(cat)
    print("---")


Results for soldiers
ADF Statistic: -7.941174847760078
p-value: 3.3108120354573906e-12
---
Results for tanks
ADF Statistic: -3.440495478878814
p-value: 0.009653452654064289
---
Results for armveh
ADF Statistic: -4.110545477911234
p-value: 0.0009303734864616723
---
Results for artillery
ADF Statistic: -7.96848306496118
p-value: 2.8227211298044534e-12
---
Results for mlrs
ADF Statistic: -8.757767824102986
p-value: 2.7258503612284205e-14
---
Results for airdef
ADF Statistic: -8.003259797482595
p-value: 2.303521283522486e-12
---
Results for aircraft
ADF Statistic: -5.081027714478858
p-value: 1.5304172398541343e-05
---
Results for helicopters
ADF Statistic: -1.3450534918973491
p-value: 0.6083167041317705
---
Results for drones
ADF Statistic: -1.6071160968763376
p-value: 0.48001794415091764
---
Results for vehicles
ADF Statistic: -8.076259506149201
p-value: 1.5026865468874368e-12
---
Results for boats
ADF Statistic: -9.146402339134163
p-value: 2.7604460304806163e-15
---
Results for specequip

# Casualties-weather models

In [ ]:
mod1 = smf.ols(formula=f'boats~temp', data=war_weath)
res1 = mod1.fit()
print(res1.summary())

                            OLS Regression Results                            
Dep. Variable:                  boats   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     3.769
Date:                Wed, 01 Jun 2022   Prob (F-statistic):             0.0555
Time:                        17:02:29   Log-Likelihood:                -50.685
No. Observations:                  87   AIC:                             105.4
Df Residuals:                      85   BIC:                             110.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1377      0.047      2.907      0.0

In [ ]:
mod2 = smf.ols(formula=f'drones~cloud', data=war_weath)
res2 = mod2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                 drones   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     16.83
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           9.35e-05
Time:                        17:02:29   Log-Likelihood:                -263.22
No. Observations:                  87   AIC:                             530.4
Df Residuals:                      85   BIC:                             535.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.2152      1.075      8.569      0.0

In [ ]:
mod3 = smf.ols(formula=f'armveh~cloud', data=war_weath)
res3 = mod3.fit()
print(res3.summary())

                            OLS Regression Results                            
Dep. Variable:                 armveh   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     5.680
Date:                Wed, 01 Jun 2022   Prob (F-statistic):             0.0194
Time:                        17:02:29   Log-Likelihood:                -471.77
No. Observations:                  87   AIC:                             947.5
Df Residuals:                      85   BIC:                             952.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.7562     11.819      0.995      0.3

# Phases hypothesis testing

In [ ]:
diff["phase_one"] = np.where(diff.index < "14.03.2022", 1, 0)
diff["phase_two"] = np.where(("14.03.2022" < diff.index) & (diff.index < "19.04.2022"), 1, 0)

In [ ]:
mod4 = smf.ols(formula=f'soldiers~phase_one', data=diff)
res4 = mod4.fit()
print(res4.summary())

                            OLS Regression Results                            
Dep. Variable:               soldiers   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     14.60
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           0.000252
Time:                        17:02:29   Log-Likelihood:                -652.34
No. Observations:                  87   AIC:                             1309.
Df Residuals:                      85   BIC:                             1314.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    243.5714     52.803      4.613      0.0

In [ ]:
mod5 = smf.ols(formula=f'tanks~phase_one', data=diff)
res5 = mod5.fit()
print(res5.summary())

                            OLS Regression Results                            
Dep. Variable:                  tanks   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     9.746
Date:                Wed, 01 Jun 2022   Prob (F-statistic):            0.00246
Time:                        17:02:29   Log-Likelihood:                -319.84
No. Observations:                  87   AIC:                             643.7
Df Residuals:                      85   BIC:                             648.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0143      1.156     11.261      0.0

In [ ]:
mod6 = smf.ols(formula=f'armveh~phase_one', data=diff)
res6 = mod6.fit()
print(res6.summary())

                            OLS Regression Results                            
Dep. Variable:                 armveh   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     9.269
Date:                Wed, 01 Jun 2022   Prob (F-statistic):            0.00310
Time:                        17:02:29   Log-Likelihood:                -470.08
No. Observations:                  87   AIC:                             944.2
Df Residuals:                      85   BIC:                             949.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     27.3571      6.499      4.210      0.0

# Correlations and Predictions

# Finding best prediction model for each category

In [ ]:
NLAGS=6
diff = diff.astype('float32')

def train_test_split(series=None, nlags=NLAGS):
    return series[:-nlags], series[-nlags:]

In [ ]:
from statsmodels.tsa.stattools import adfuller

def get_predictable_categories(df=diff, nlags=NLAGS):
    predictable_categories = []
    for category in MAIN_CATEGORIES:
        ACF, PACF = acf(df[category], nlags=nlags)[1:], pacf(df[category], nlags=nlags)[1:]
        adf, p_value, *other = adfuller(df[category], autolag='AIC')

        if (ACF > .2).any() and (PACF > .2).any() and p_value < 0.05:
            predictable_categories.append(category)

    return predictable_categories

predictable_categories = get_predictable_categories(diff)
predictable_categories

['soldiers', 'tanks', 'armveh']

In this function I am checking the following conditions:
1. Akaike Information Criterion (AIC) is minimal;
2. Ljung-Box test of autocorrelation in residuals;

The Ljung-Box test is made to test null hypothesis: a set of residual autocorrelations is zero.

In [ ]:
def get_best_orders(*categories, df=diff, nlags=NLAGS):
    dff = df.copy().astype('float32')
    best_orders = dict.fromkeys(categories, (0, 0))
    for category in categories:
        print(f'Processing {category} category...')
        least_aic, least_resid = float('inf'), float('inf')
        train, test = train_test_split(dff[category], nlags)

        for ar_coef in range(nlags):
            for ma_coef in range(nlags):
                try: 
                    best_model = ARIMA(endog=train, order=(ar_coef, 0, ma_coef)).fit()
                    aic = best_model.aic
                    ljung_box, p_value = acorr_ljungbox(best_model.resid)
                    # pred, stderr, conf_int = best_model.forecast(nlags)
                    # resid = (abs(pred - test)).sum()
                    #  and resid < least_resid
                    if aic < least_aic and (p_value >= .05).all():
                        best_orders[category] = (ar_coef, ma_coef)
                        least_aic = aic
                except:
                    continue
            
        output.clear()
    return best_orders

try:
    print(best_orders)
except:
    best_orders = get_best_orders(*predictable_categories, df=diff)
    print(best_orders)

{'soldiers': (1, 1), 'tanks': (2, 0), 'armveh': (0, 3)}


Limit the MA coeff < 4.
Laf of 5 could be explained by day-frequency.

In [ ]:
def filter_orders(best_orders):
    return {category: coeffs for category, coeffs in best_orders.items() if any(coef for coef in coeffs)}

best_orders = filter_orders(best_orders)
best_orders['soldiers'] = (1,1)
best_orders

{'armveh': (0, 3), 'soldiers': (1, 1), 'tanks': (2, 0)}

In [ ]:
def get_predictions(best_orders, df=diff, nlags=NLAGS):
    predictions = dict.fromkeys(best_orders.keys())
    for category, (ar_coef, ma_coef) in best_orders.items():
        train, test = train_test_split(df[category])
        arma_model = ARIMA(train, order=(ar_coef, 0, ma_coef)).fit()
        pred, stderr, conf_int = arma_model.forecast(nlags)
        predictions[category] = (test, pred)
    return predictions, test.index

predictions, x_axis = get_predictions(best_orders, df=diff)
# predictions

### plot_predictions()

In [ ]:
def plot_predictions(predictions, x_axis=None):
    rows, cols, counter = (len(predictions) + 1) // 3, 3, 0
    colors = {'actual': 'red', 'test': 'black'}
    names = list(predictions.keys())
    figure = make_subplots(rows=rows, cols=cols, subplot_titles=names)
    for category, (test, pred) in predictions.items():
        row, col = counter // cols + 1, counter % cols + 1
        counter += 1

        fig = go.Scatter(
            x=x_axis, y=test, name='Actual', legendgroup = counter,
            line=dict(color='red', width=4)
        )

        figure.add_trace(fig, row=row, col=col)
        
        fig = go.Scatter(x=x_axis, y=pred, name='Predicted', legendgroup = counter,
                         line=dict(color='black', width=3, dash='dot')
        )
        figure.add_trace(fig, row=row, col=col)

    figure.update_layout(height=300, width=1100, legend_tracegroupgap=1000,
                         title_text=f'ARMA prediction', title_x=.5)
    return figure

figure = plot_predictions(predictions, x_axis=x_axis)
figure.show()

# Improving model with weather conditions

In [ ]:
without_north_columns = [col for col in data.columns
                         if col.split('_')[0] not in NORTH_REGIONS]
data = data[without_north_columns]

main_weather_columns = [col for col in all_weather
                        if col.split('_')[0] in MAIN_REGIONS]
weather = all_weather.iloc[11:,:][main_weather_columns]
weather = sm.add_constant(weather)
data.shape, weather.shape

((87, 33), (87, 21))

In [ ]:
def control_target_split(data, best_orders):
    target = list(best_orders.keys())
    Y = data[target]
    control = set(data.columns).difference(set(target))
    X = data[control]
    return X, Y

X, Y = control_target_split(data, best_orders)
X.shape, Y.shape

((87, 30), (87, 3))

In [ ]:
def get_correlation_significant_regressors(X, target, threshold=.2):
    significant = {'const'}
    for control in X.columns:
            corr = np.corrcoef(X[control], target)[1, 0]
            if abs(corr) > threshold:
                significant.add(control)
    return significant

In [ ]:
def get_pvalue_significant_regressors(X, target, threshold=.1):
    significant = set()
    fitted = sm.OLS(target, X).fit()
    for pvalue, param in zip(fitted.pvalues, X.columns):
        if pvalue < threshold:
            significant.add(param)
    return significant

In [ ]:
def train_model(X, target, significant, threshold=.1):
    while True:
        fitted = sm.OLS(target, X[significant]).fit()
        still_significant = {'const'}
        for pvalue, param in zip(fitted.pvalues, significant):
            if pvalue <= threshold:
                still_significant.add(param)
        if len(significant) == len(still_significant):
            break
        else:
            significant = still_significant.copy()
    return significant

In [ ]:
def get_significant_regressors(X, Y):
    significant_regressors = dict.fromkeys(Y.columns)
    for target in Y.columns:
        significant = get_correlation_significant_regressors(X, Y[target])
        significant = significant.union(get_pvalue_significant_regressors(X, Y[target]))
        significant = train_model(X[significant], Y[target], significant)
    
        fitted = sm.OLS(Y[target], X[significant]).fit()
        # print(fitted.summary())

        significant_regressors[target] = significant
    return significant_regressors

significant_regressors = get_significant_regressors(weather, Y)
for k, v in significant_regressors.items():
    print(k)
    print(v)

soldiers
{'kharkiv_temp', 'melitopol_temp', 'donetsk_temp', 'izyum_temp', 'izyum_wind', 'const'}
tanks
{'kherson_pres', 'donetsk_pres', 'kharkiv_pres', 'const', 'izyum_pres'}
armveh
{'donetsk_pres', 'const'}


In [ ]:
def get_weather_predictions(df, significant):
    predictions = dict.fromkeys(significant.keys())
    for category, regressors in significant.items():
        train, test = train_test_split(df[category])
        fitted = sm.OLS(train, df[regressors].iloc[:-6,:]).fit()
        pred = fitted.predict(df[regressors].iloc[-6:,:])
        predictions[category] = (test, pred)
    return predictions, test.index

weather_predictions, x_axis = get_weather_predictions(data, significant_regressors)

In [ ]:
def plot_predictions(predictions, x_axis=None):
    rows, cols, counter = (len(predictions) + 1) // 3, 3, 0
    colors = {'actual': 'red', 'test': 'black'}
    names = list(predictions.keys())
    figure = make_subplots(rows=rows, cols=cols, subplot_titles=names)
    for category, (test, pred) in predictions.items():
        row, col = counter // cols + 1, counter % cols + 1
        counter += 1

        fig = go.Scatter(
            x=x_axis, y=test, name='Actual', legendgroup = counter,
            line=dict(color='red', width=4)
        )

        figure.add_trace(fig, row=row, col=col)
        
        fig = go.Scatter(x=x_axis, y=pred, name='Predicted', legendgroup = counter,
                         line=dict(color='black', width=3, dash='dot')
        )
        figure.add_trace(fig, row=row, col=col)

    figure.update_layout(height=300, width=1100, legend_tracegroupgap=1000,
                         title_text=f'Linear regression prediction', title_x=.5)
    return figure

figure = plot_predictions(weather_predictions, x_axis=x_axis)
figure.show()

Спробувати F-test по містах або по групах чинників (тестую вітер в усіх регіонах)

# Improving model with war phases

In [ ]:
phase_one, phase_two = "2022-03-01 00:00:00", "2022-05-12 00:00:00"

data['phase_one'] = np.where(data.index < phase_one, 1, 0)
data['phase_two'] = np.where((data.index >= phase_one) & (data.index < phase_two), 1, 0)

fitted = sm.OLS(data['tanks'], data[['const', 'phase_one', 'phase_two']]).fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                  tanks   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.429
Method:                 Least Squares   F-statistic:                     33.26
Date:                Wed, 01 Jun 2022   Prob (F-statistic):           2.29e-11
Time:                        17:02:29   Log-Likelihood:                -299.19
No. Observations:                  87   AIC:                             604.4
Df Residuals:                      84   BIC:                             611.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.9091      2.313      3.852      0.0

In [ ]:
def get_final_predictions(best_orders, significant, df, nlags=NLAGS):
    predictions = dict.fromkeys(best_orders.keys())
    for category, (ar_coef, ma_coef) in best_orders.items():
        train, test = train_test_split(df[category])
        selected = significant[category].copy().union(['phase_one', 'phase_two'])
        final_model = SARIMAX(train, order=(ar_coef, 0, ma_coef),
                            exog=df[selected].iloc[:-6,:],
                            trend='c').fit()
        pred = final_model.forecast(nlags, exog=df[selected].iloc[-6:,:])
        predictions[category] = (test, pred)
    return predictions, test.index

predictions, x_axis = get_final_predictions(best_orders, significant_regressors, data)
# predictions

In [ ]:
def plot_predictions(predictions, x_axis=None):
    rows, cols, counter = (len(predictions) + 1) // 3, 3, 0
    colors = {'actual': 'red', 'test': 'black'}
    names = list(predictions.keys())
    figure = make_subplots(rows=rows, cols=cols, subplot_titles=names)
    for category, (test, pred) in predictions.items():
        row, col = counter // cols + 1, counter % cols + 1
        counter += 1

        fig = go.Scatter(
            x=x_axis, y=test, name='Actual', legendgroup = counter,
            line=dict(color='red', width=4)
        )

        figure.add_trace(fig, row=row, col=col)
        
        fig = go.Scatter(x=x_axis, y=pred, name='Predicted', legendgroup = counter,
                         line=dict(color='black', width=3, dash='dot')
        )
        figure.add_trace(fig, row=row, col=col)

    figure.update_layout(height=300, width=1100, legend_tracegroupgap=1000,
                         title_text=f'ARMA with OLS regressors', title_x=.5)
    return figure

figure = plot_predictions(predictions, x_axis=x_axis)
figure.show()

# Compare models

In [ ]:
arma_pred, x_axis = get_predictions(best_orders, df=diff)
weather_pred, x_axis = get_weather_predictions(data, significant_regressors)
final_pred, x_axis = get_final_predictions(best_orders, significant_regressors, data)

In [ ]:
errors = dict.fromkeys(arma_pred.keys())
for cat, (test, pred) in arma_pred.items():
    mae = round(abs(test - pred).sum(), 2)
    ssr = round(((test - pred) ** 2).sum(), 2)
    errors[cat] = (mae, ssr)
print(errors)

{'soldiers': (764.26, 107767.01), 'tanks': (29.34, 196.2), 'armveh': (83.08, 1176.6)}


In [ ]:
errors = dict.fromkeys(weather_pred.keys())
for cat, (test, pred) in weather_pred.items():
    mae = round(abs(test - pred).sum(), 2)
    ssr = round(((test - pred) ** 2).sum(), 2)
    errors[cat] = (mae, ssr)
print(errors)

{'soldiers': (546.04, 89093.72), 'tanks': (18.98, 93.19), 'armveh': (73.59, 1342.2)}


In [ ]:
errors = dict.fromkeys(final_pred.keys())
for cat, (test, pred) in final_pred.items():
    mae = round(abs(test - pred).sum(), 2)
    ssr = round(((test - pred) ** 2).sum(), 2)
    errors[cat] = (mae, ssr)
print(errors)

{'soldiers': (528.54, 68157.42), 'tanks': (20.83, 132.81), 'armveh': (150.27, 4324.12)}
